In [ ]:
import numpy as np
import multiprocessing as mp
import random,copy,string
from nltk.tokenize import word_tokenize
from scipy.stats import pearsonr
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Convolution1D, MaxPooling1D, Flatten
from tensorflow.python.keras.layers import Lambda, multiply, concatenate, Dense
from tensorflow.python.keras.regularizers import l2
from tensorflow.python.keras.callbacks import Callback
from tensorflow.python.keras.preprocessing.sequence import pad_sequences


In [ ]:
import os, string, random, time, math
import torch.optim as optim
import matplotlib.pyplot as plt
from IPython.display import clear_output
import torch

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
'''from google.colab import drive
drive.mount('/content/drive/')'''

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
#cd 'drive/My Drive/'

/content/drive/My Drive


In [ ]:
#ls


 2017_takelab/                        model_cifar.pt
 2017_takelab+cnn/                    msr_vid_SVR.ipynb
'Colab Notebooks'/                    name2lang.txt
 data.zip                             processed_takelab_glove.txt
 deep_fasttext_epoch.pt               processed_takelab_jc.txt
 fasttext_cross0                      processed_takelab_lin.txt
 fasttext_cross1                      processed_takelab_output_MSRpar.txt
 fasttext_cross2                      processed_wv_wwv.txt
 fasttext_cross_epoch.pt              pytorch_cnn_code.ipynb
 fasttext_cross_model.pt              sts-dev.csv
 fasttext_cross.pt                    STS_survey.odt
 fasttext_kl_adamax_model.pt          sts-test.csv
 fasttext_kl_adam_model.pt            sts-train.csv
 fasttext_kl_epoch.pt                 substitute_vec.txt
 glove.840B.300d.txt                  takelab_glove.txt
 glove_ce_epoch.pt                    takelab_jc.txt
 glove_ce_model.pt                    takelab_lin.txt
 glove_cross_2FNN_epoch.pt

In [ ]:
c = dict()
c['num_runs']   = 3
c['num_epochs'] = 2
c['num_batchs'] = 4
c['batch_size'] = 3
c['wordvectdim']  = 300
c['sentencepad']  = 60
c['num_classes']  = 6
c['cnnfilters']     = {1: 1800}
c['cnninitial']     = 'he_uniform'
c['cnnactivate']    = 'relu'
c['densedimension'] = list([1800])
c['denseinitial']   = 'he_uniform'
c['denseactivate']  = 'tanh'
c['optimizer']  = 'adam'

In [ ]:
wordvectdim=300
wordtoindex= dict()
indextovector= []
indextovector.append(np.zeros(wordvectdim))    

# Loading word vectors

In [ ]:
'''dictname= 'substitute_vec.txt'
lines = open('drive/My Drive/'dictname,'r',encoding='latin1').readlines()'''

"dictname= 'substitute_vec.txt'\nlines = open('drive/My Drive/'dictname,'r',encoding='latin1').readlines()"

In [ ]:
lines= (open('../glove.840B.300d.txt','r').readlines())

In [ ]:
def _worker(args):
        print(args[1])
        wordtoindex   = dict()
        indextovector = []
        for line in args[0]:
            elements = line.split(' ')
            wordtoindex[elements[0]] = len(indextovector)+args[1]+1    # index of first word starts with 1
            indextovector.append(np.array(elements[1:]).astype(float))
        return (wordtoindex,indextovector)

In [ ]:
print("Number of cpu : ", mp.cpu_count())

Number of cpu :  4


In [ ]:
blocksize=500
r_list = mp.Pool(4).map(_worker, ((lines[block:block+blocksize], block) for block in range(0,len(lines),blocksize)))

0
500
1000
1500
2000
2500
3000
3500
4000
4500
137500
5000
138000
5500
138500
6000
139000
6500
7000
139500
7500
140000
275000
8000
275500
140500
8500
276000
141000
9000
276500
141500
9500
277000
142000
10000
277500
142500
278000
10500
143000
278500
11000
143500
279000
412500
11500
144000
279500
413000
12000
144500
413500
280000
12500
414000
145000
280500
13000
414500
145500
281000
13500
415000
146000
281500
14000
415500
146500
282000
14500
416000
282500
147000
15000
416500
283000
147500
15500
417000
283500
148000
16000
417500
284000
148500
418000
16500
284500
149000
418500
17000
285000
149500
419000
17500
285500
150000
419500
18000
286000
150500
420000
18500
151000
286500
420500
19000
151500
287000
421000
19500
287500
152000
421500
20000
288000
152500
422000
20500
153000
288500
422500
21000
289000
153500
423000
21500
289500
154000
423500
22000
290000
154500
424000
22500
290500
155000
424500
23000
291000
155500
425000
23500
291500
156000
425500
24000
292000
156500
426000
24500
292500
157

In [ ]:
for r in r_list:
          wordtoindex.update(r[0])
          indextovector.extend(r[1])

In [ ]:
indextovector = np.array(indextovector, dtype='float32')

In [ ]:
indextovector.shape

(2196018, 300)

# Params

In [ ]:
class Params(object):
  def __init__(self, batch_size, epochs, seed, log_interval):
    self.batch_size = batch_size
    self.epochs = epochs
    self.seed = seed
    self.log_interval = log_interval

args= Params(16,8,0,8)

# Load Data

In [ ]:
def matrixize(sentencelist, sentencepad):
        indexlist = []
        for sentence in sentencelist:
            indexes = []
            for word in sentence:
                word = word.lower()
                if word not in wordtoindex: indexes.append(1)
                else: indexes.append(wordtoindex[word])
            indexlist.append(indexes)
        return indextovector[(pad_sequences(indexlist, maxlen=sentencepad, truncating='post', padding='post'))]

In [ ]:
def _load_data(filename):
        s0,s1,labels = [],[],[]
        lines=open(filename,'r').read().splitlines()
        for line in lines:
            _,_,_,_, label, s0x, s1x = line.rstrip().split('\t')[:7]
            labels.append(float(label))
            s0.append([word.lower() for word in word_tokenize(s0x) if word not in string.punctuation])
            s1.append([word.lower() for word in word_tokenize(s1x) if word not in string.punctuation])
        m0 = matrixize(s0, c['sentencepad'])
        m1 = matrixize(s1, c['sentencepad'])
        classes = np.zeros((len(labels),c['num_classes']))
        for i, label in enumerate(labels): # making probability distribution of classes
            if np.floor(label) + 1 < c['num_classes']:
                classes[i, int(np.floor(label)) + 1] = label - np.floor(label)
            classes[i, int(np.floor(label))] = np.floor(label) - label + 1
        return {'labels': labels, 's0': s0, 's1': s1, 'classes': classes, 'm0': m0, 'm1': m1}


In [ ]:
'''trainload= _load_data('sts-train.csv')
validation= _load_data('sts-dev.csv')
len(trainload['s0'])'''

"trainload= _load_data('sts-train.csv')\nvalidation= _load_data('sts-dev.csv')\nlen(trainload['s0'])"

In [ ]:
'''data_len= []
for i in trainload['s0']:
  data_len.append(len(i))
traiload_maxsent=max(data_len)
traiload_maxsent'''

"data_len= []\nfor i in trainload['s0']:\n  data_len.append(len(i))\ntraiload_maxsent=max(data_len)\ntraiload_maxsent"

In [ ]:
'''data_len= []
for i in  validation['s0']:
  data_len.append(len(i))
validation_maxsent=max(data_len)
validation_maxsent'''

"data_len= []\nfor i in  validation['s0']:\n  data_len.append(len(i))\nvalidation_maxsent=max(data_len)\nvalidation_maxsent"

# Sampling Batch

In [ ]:
def _sample_pairs(data,batch_size):
  datacopy={}
  for i in data.keys():
    datacopy[i]= []
  for i in range(batch_size):
    index = np.random.randint(len(data['labels']))
    #print(index)
    for key,value in data.items():
      datacopy[key].append(value[index])
  datacopy['classes']= torch.tensor(datacopy['classes'])
  datacopy['m0']= torch.tensor(datacopy['m0'])
  datacopy['m1']= torch.tensor(datacopy['m1'])
  return datacopy

# Defining CNN Model

In [ ]:
import torch.nn as nn

In [ ]:
class CNN_Model(nn.Module):
  def __init__(self,nh=1800):
    super(CNN_Model, self).__init__()
    self.cnn= nn.Sequential(nn.Conv1d(300,1800,1),              #(bs,300,60) -> (bs, 1800, 60)
                            #nn.BatchNorm1d(1800),
                            nn.ReLU(),
                            #torch.nn.Dropout(0.2),
                            nn.MaxPool1d(kernel_size=60))       #(bs,1800,60)-> (bs, 1800, 1)

    self.Linear= nn.Sequential(nn.Linear(3600,nh),            #(bs, 3600)  -> (bs, 1800 )
                               nn.Tanh(),
                               nn.Linear(nh,6),               #(bs,1800)   -> (bs,6)
                               nn.Softmax(dim=1))
                            
    

  def forward(self,input1, input2):
    input1= self.cnn(input1)
    input2= self.cnn(input2)
    #print("input1=",input1.shape)
    #print("input2=",input2.shape)
    input1= torch.flatten(input1,1)
    input2= torch.flatten(input2,1)
    #print("input1=",input1.shape)
    #print("input2=",input2.shape)
    absdiff= abs(input1 - input2)
    mulDifference= input1 * input2
    concatenate = torch.cat((absdiff,mulDifference),1)
    #print(concatenate.shape)
    output= self.Linear(concatenate)
    #print(output.shape)
    return (output)

In [ ]:
def weights_init(m):
    if isinstance(m, nn.Conv1d) or isinstance(m, torch.nn.Linear):
        torch.nn.init.kaiming_normal_(m.weight)
        torch.nn.init.zeros_(m.bias)


# Defining Loss

In [ ]:
def _lossfunction(y_true,y_pred):
    
    #print("y_true.shape",y_true.shape)
    #print("y_pred.shape",y_pred.shape)
    ny_true = y_true[:,1] + 2*y_true[:,2] + 3*y_true[:,3] + 4*y_true[:,4] + 5*y_true[:,5]
    #print(ny_true)
    ny_pred = y_pred[:,1] + 2*y_pred[:,2] + 3*y_pred[:,3] + 4*y_pred[:,4] + 5*y_pred[:,5]
    #print(ny_pred)
    my_true = (ny_true.mean())
    #print(my_true)
    my_pred = (ny_pred.mean())
    #print(my_pred)
    var_true = (ny_true - my_true)**2
    var_pred = (ny_pred - my_pred)**2
    eps= 1e-14
    return -torch.sum((ny_true-my_true)*(ny_pred-my_pred),axis=-1) / (torch.sqrt(torch.sum(var_true,axis=-1)* torch.sum(var_pred,axis=-1)+eps))

# Training

## Train setup

In [ ]:
def train_batch(net,trainload, opt,batch_size, device= 'cpu'):
  
  net.train().to(device)
  opt.zero_grad()
  
  

  data= _sample_pairs(trainload,batch_size)
  input1= (torch.transpose(data['m0'],1,2)).to(device)
  input2= (torch.transpose(data['m1'],1,2)).to(device)
  true_output= (data['classes']).to(device, dtype=torch.float32)
  pred_output= net(input1,input2).to(torch.float32)
  loss= _lossfunction(true_output,pred_output)

  #loss= criterion(pred_output,true_output)
  loss.backward()
  opt.step()
  #print("lossgf",loss.item())
  #return loss.item()
  return loss

## Validation Setup

In [ ]:
def valid_batch(net,validation, opt,batch_size, device= 'cpu'):
  net.eval().to(device)
  
  data = _sample_pairs(validation,batch_size)
  input1= (torch.transpose(data['m0'],1,2)).to(device)
  input2= (torch.transpose(data['m1'],1,2)).to(device)
  true_output= (data['classes']).to(device, dtype=torch.float32)
  pred_output= net(input1,input2).to(torch.float32)
  loss= _lossfunction(true_output,pred_output)
  #loss= criterion(pred_output,true_output)
  #return loss.item()
  return loss


# Full Training Setup

In [ ]:
def train_setup_Adam(net,lr = 0.01,  batch_size = args.batch_size, momentum = 0.9, device = 'cpu',epoch= args.epochs):
  torch.autograd.set_detect_anomaly(True)
  net.apply(weights_init)
  valid_loss_min = np.Inf
  opt = optim.Adam(net.parameters(), lr=lr)
  trainload= _load_data('sts-train.csv')
  validation= _load_data('sts-dev.csv')
  train_n_batch= int(len(trainload['labels'])/batch_size)
  train_loss_arr = np.zeros(train_n_batch+ 1)
  ###################
  # train the model #
  ###################
  
  for epoch in range(epoch):
    if epoch != 0:
      net.load_state_dict(torch.load('glove_cross_epoch.pt'))
    for i in range(train_n_batch):
      train_loss_arr[i+1] = (train_loss_arr[i]*i + train_batch(net,trainload, opt, batch_size, device))/(i + 1)
      #print("train_loss_MSE",train_loss_arr[i+1])

     
   
    
      #if i%display_freq == display_freq-1:
      '''clear_output(wait=True)
      print('Iteration', i, 'Loss', loss_arr[i])
      plt.figure()
      plt.plot(train_loss_arr,'-*')
      plt.xlabel('Iteration')
      plt.ylabel('Loss')
      plt.show()
      print('\n\n')'''

      ######################    
      # validate the model #
      ######################
    valid_n_batch = int(len(validation['labels'])/batch_size)
    valid_loss_arr = np.zeros(valid_n_batch + 1)
    for i in range(valid_n_batch):
      valid_loss_arr[i+1] = (valid_loss_arr[i]*i + valid_batch(net,validation, opt, batch_size, device))/(i + 1)
    
    #mlflow.log_metric('train_loss_KLDIV', train_loss_arr[i+1])
    #mlflow.log_metric('valid_loss_KLDIV', valid_loss_arr[i+1])

    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
          epoch, train_loss_arr[train_n_batch ], valid_loss_arr[valid_n_batch ]))
  
    if valid_loss_arr[valid_n_batch ] <= valid_loss_min:
          print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
          valid_loss_min, valid_loss_arr[valid_n_batch ]))
          torch.save(net.state_dict(), 'glove_cross_epoch.pt')
          valid_loss_min = valid_loss_arr[valid_n_batch ]


In [ ]:
def train_setup_Adamax(net,lr = 0.01,  batch_size = args.batch_size, momentum = 0.9, device = 'cpu',epoch= args.epochs):
  torch.autograd.set_detect_anomaly(True)
  valid_loss_min = np.Inf
  net.apply(weights_init)
  opt = optim.Adamax(net.parameters(), lr=lr)
  trainload= _load_data('sts-train.csv')
  validation= _load_data('sts-dev.csv')
  train_n_batch= int(len(trainload['labels'])/batch_size)
  train_loss_arr = np.zeros(train_n_batch+ 1)
  ###################
  # train the model #
  ###################
  
  for epoch in range(epoch):
    if epoch != 0:
      net.load_state_dict(torch.load('glove_cross_epoch.pt'))
    for i in range(train_n_batch):
      train_loss_arr[i+1] = (train_loss_arr[i]*i + train_batch(net,trainload, opt, batch_size, device))/(i + 1)
      #print("train_loss_MSE",train_loss_arr[i+1])

     
   
    
      #if i%display_freq == display_freq-1:
      '''clear_output(wait=True)
      print('Iteration', i, 'Loss', loss_arr[i])
      plt.figure()
      plt.plot(train_loss_arr,'-*')
      plt.xlabel('Iteration')
      plt.ylabel('Loss')
      plt.show()
      print('\n\n')'''

      ######################    
      # validate the model #
      ######################
    valid_n_batch = int(len(validation['labels'])/batch_size)
    valid_loss_arr = np.zeros(valid_n_batch + 1)
    for i in range(valid_n_batch):
      valid_loss_arr[i+1] = (valid_loss_arr[i]*i + valid_batch(net,validation, opt, batch_size, device))/(i + 1)
    
    #mlflow.log_metric('train_loss_KLDIV', train_loss_arr[i+1])
    #mlflow.log_metric('valid_loss_KLDIV', valid_loss_arr[i+1])

    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
          epoch, train_loss_arr[train_n_batch ], valid_loss_arr[valid_n_batch ]))
  
    if valid_loss_arr[valid_n_batch ] <= valid_loss_min:
          print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
          valid_loss_min, valid_loss_arr[valid_n_batch ]))
          torch.save(net.state_dict(), 'glove_cross_epoch.pt')
          valid_loss_min = valid_loss_arr[valid_n_batch ]


In [ ]:
'''net= CNN_Model()
train_setup(net,lr = 0.05, momentum = 0.95, device='cpu')'''


"net= CNN_Model()\ntrain_setup(net,lr = 0.05, momentum = 0.95, device='cpu')"

# Evaluate

In [ ]:
'''net.load_state_dict(torch.load('model_cifar.pt'))'''

"net.load_state_dict(torch.load('model_cifar.pt'))"

## Evaluation Setup

In [ ]:
def eval_setup(net,batch_size = 1379, device = 'cpu'):
  
  
  net.load_state_dict(torch.load('glove_cross_epoch.pt'))
  net.eval().to(device)
  testload= _load_data('sts-test.csv')
  test_n_batch= int(len(testload['labels'])/batch_size)
  #print(len(testload['labels']))
  test_loss_arr = np.zeros(test_n_batch+ 1)


  data= _sample_pairs(testload,batch_size)
  input1= (torch.transpose(data['m0'],1,2)).to(device)
  input2= (torch.transpose(data['m1'],1,2)).to(device)
  pred_output= (net(input1,input2)).to('cpu')
  pred= pred_output.detach().numpy()
  prediction = np.dot(np.array(pred),np.arange(c['num_classes']))
  print('pred_shape',prediction.shape)
  goldlabels = data['labels']
  result=round((pearsonr(prediction, goldlabels)[0]),4)
  print("result",result)

  return result


In [ ]:
'''net=CNN_Model(48)
eval_setup(net,device = 'cuda:0')'''

"net=CNN_Model(48)\neval_setup(net,device = 'cuda:0')"

# Hyper-parameter Tuning

## optim.SGD()

In [ ]:

'''for lrng in [0.01,0.001]:
    for mntm in [0.09,0.1,0.2]:
        for hidden_nodes in [1800]:
        
          expt_id = '%d_%d_%d' % (int(lrng*100), int(mntm*100), hidden_nodes)
          print('\nLR = %.5f, Momentum = %.5f, Hidden nodes = %d\n' % (lrng, mntm, hidden_nodes))

          model = CNN_Model(hidden_nodes)

          '''with mlflow.start_run(nested=True) as run:

            for key, value in vars(args).items():
              mlflow.log_param(key, value)
            mlflow.log_param('lr', lrng)
            mlflow.log_param('momentum', mntm)
            mlflow.log_param('hidden_nodes', hidden_nodes)'''

          train_setup(model,lr=lrng,momentum= mntm,device='cuda:0')

          eval_setup(model,device = 'cuda:0')'''
  


SyntaxError: ignored

## optim.adam

In [ ]:
best_model_score = -(np.Inf)
for learning_rate in [0.0001,0.0005,0.0006]:
    
        for hidden_nodes in [1600,1800,2400]:
        
          expt_id = '%d_%d' % (int(learning_rate*100),  hidden_nodes)
          print('\nLR = %.8f, Hidden nodes = %d\n' % (learning_rate, hidden_nodes))

          model = CNN_Model(hidden_nodes)
          train_setup_Adam(model,lr=learning_rate,device='cuda:0')

          eval_score=eval_setup(model,device = 'cuda:0')

          if eval_score > best_model_score :
            print('best_model_score increased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            best_model_score, eval_score))
            torch.save(model.state_dict(), 'glove_cross_adam_model.pt')
            best_model_score= eval_score


LR = 0.00010000, Hidden nodes = 1600

Epoch: 0 	Training Loss: -0.724155 	Validation Loss: -0.759911
Validation loss decreased (inf --> -0.759911).  Saving model ...
Epoch: 1 	Training Loss: -0.839455 	Validation Loss: -0.768247
Validation loss decreased (-0.759911 --> -0.768247).  Saving model ...
Epoch: 2 	Training Loss: -0.893974 	Validation Loss: -0.811878
Validation loss decreased (-0.768247 --> -0.811878).  Saving model ...
Epoch: 3 	Training Loss: -0.922450 	Validation Loss: -0.797742
Epoch: 4 	Training Loss: -0.922223 	Validation Loss: -0.795153
Epoch: 5 	Training Loss: -0.920373 	Validation Loss: -0.790881
Epoch: 6 	Training Loss: -0.917072 	Validation Loss: -0.804178
Epoch: 7 	Training Loss: -0.917823 	Validation Loss: -0.802283
pred_shape (1379,)
result 0.7613
best_model_score increased (-inf --> 0.761300).  Saving model ...

LR = 0.00010000, Hidden nodes = 1800

Epoch: 0 	Training Loss: -0.751870 	Validation Loss: -0.778110
Validation loss decreased (inf --> -0.778110).  S

In [ ]:
best_model_score = -(np.Inf)
for learning_rate in [0.0001,0.0005,0.0006]:
    
        for hidden_nodes in [1600,1800,1700]:
        
          expt_id = '%d_%d' % (int(learning_rate*100),  hidden_nodes)
          print('\nLR = %.8f, Hidden nodes = %d\n' % (learning_rate, hidden_nodes))

          model = CNN_Model(hidden_nodes)
          train_setup_Adamax(model,lr=learning_rate,device='cuda:0')

          eval_score=eval_setup(model,device = 'cuda:0')

          if eval_score >= best_model_score :
            print('best_model_score increased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            best_model_score, eval_score))
            torch.save(model.state_dict(), 'glove_cross_adamax_model.pt')
            best_model_score= eval_score


LR = 0.00010000, Hidden nodes = 1600

Epoch: 0 	Training Loss: -0.737807 	Validation Loss: -0.781203
Validation loss decreased (inf --> -0.781203).  Saving model ...
Epoch: 1 	Training Loss: -0.826764 	Validation Loss: -0.809836
Validation loss decreased (-0.781203 --> -0.809836).  Saving model ...
Epoch: 2 	Training Loss: -0.869590 	Validation Loss: -0.806272
Epoch: 3 	Training Loss: -0.873411 	Validation Loss: -0.795774
Epoch: 4 	Training Loss: -0.870513 	Validation Loss: -0.794509
Epoch: 5 	Training Loss: -0.864493 	Validation Loss: -0.789623
Epoch: 6 	Training Loss: -0.869042 	Validation Loss: -0.791963
Epoch: 7 	Training Loss: -0.861585 	Validation Loss: -0.792882
pred_shape (1379,)
result 0.7537
best_model_score increased (-inf --> 0.753700).  Saving model ...

LR = 0.00010000, Hidden nodes = 1800

Epoch: 0 	Training Loss: -0.735946 	Validation Loss: -0.772099
Validation loss decreased (inf --> -0.772099).  Saving model ...
Epoch: 1 	Training Loss: -0.824905 	Validation Loss: -0

# Practice

In [ ]:
trainload= _load_data('sts-train.csv')

In [ ]:
data=_sample_pairs(trainload,2)

In [ ]:
data_len= []
for i in data['s0']:
  data_len.append(len(i))
max_len_sent=max(data_len)

rep= torch.zeros(max_len_sent, 2 ,300)
rep.shape

torch.Size([9, 2, 300])

In [ ]:
(data['s0'])

[['blast', 'kills', '10', 'young', 'girls', 'in', 'eastern', 'afghanistan'],
 ['four',
  'blue',
  'and',
  'yellow',
  'planes',
  'flying',
  'over',
  'four',
  'boats']]

In [ ]:
temp=[]
rep[0][0]=torch.tensor(indextovector[wordtoindex['brown']])


In [ ]:
rep[0]

tensor([[-0.3741, -0.0763,  0.1093,  0.1866,  0.0299,  0.1827, -0.6320,  0.1331,
         -0.1290,  0.6034, -0.6804, -0.1422, -0.1336, -0.6594,  0.0524,  0.1674,
          0.6392,  1.7680,  0.3462, -0.6248, -0.1287, -0.1970, -0.3745,  0.3306,
          0.0468, -0.6535, -0.5614,  0.2274,  0.2292, -0.4158, -0.1677,  0.3354,
          0.0972, -0.4670, -0.0269, -0.0677, -0.1921, -0.1337,  0.0163, -0.2083,
          0.6486, -0.1102, -0.0510,  0.0722,  0.1877,  0.2711, -0.3142,  0.1832,
          0.3915, -0.2255, -0.3819,  0.3306, -0.0899, -0.3763,  0.0480, -0.2050,
         -0.5489,  0.3044, -0.1887, -0.3034, -0.1157, -0.3487,  0.2800,  0.0501,
         -0.2814, -0.2335, -0.3683, -0.1204, -0.2833,  0.1857,  0.1036,  0.2531,
         -0.0340,  0.1051,  0.1214, -0.1630, -0.3318,  0.1731,  0.1076, -0.9936,
         -0.1171,  0.4223,  0.1512,  0.3106,  0.2674, -0.4927,  1.8049,  1.0738,
          0.3442,  0.1128, -0.1049,  0.3694,  0.4082, -0.4037,  0.3233, -0.0939,
         -0.0290,  0.3825, -

In [ ]:
data['m0'][0][11]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [ ]:
m = nn.MaxPool1d(1)
input = torch.randn(6, 5)
input.shape

torch.Size([6, 5])

In [ ]:
input

tensor([[ 1.8696,  0.9236, -1.2735, -0.4908,  0.0378],
        [-0.2711, -1.3810,  0.3729, -1.0544,  0.3804],
        [ 0.2439,  0.7127, -0.5271,  0.8484, -1.5219],
        [-1.2225,  0.3181, -1.3479,  0.4878,  0.7140],
        [-1.1902, -0.4306, -0.3603, -1.2621,  0.3393],
        [-1.0988,  0.1381, -1.8079,  1.2449,  0.9523]])

In [ ]:
input.to(torch.float32)

tensor([[ 1.8696,  0.9236, -1.2735, -0.4908,  0.0378],
        [-0.2711, -1.3810,  0.3729, -1.0544,  0.3804],
        [ 0.2439,  0.7127, -0.5271,  0.8484, -1.5219],
        [-1.2225,  0.3181, -1.3479,  0.4878,  0.7140],
        [-1.1902, -0.4306, -0.3603, -1.2621,  0.3393],
        [-1.0988,  0.1381, -1.8079,  1.2449,  0.9523]])

In [ ]:
output = m(input)
output.shape

RuntimeError: ignored

In [ ]:
out= torch.flatten(output, 1)

In [ ]:
out.shape

In [ ]:
a=torch.tensor(np.array([1,2,3]))
b=torch.tensor(np.array([4,5,6]))
a*b

In [ ]:
c=abs(a-b)

c

In [ ]:
import math

In [ ]:
torch.cat((a,b),1).shape

In [ ]:
c= torch.randn(2, 5)
d= torch.randn(2,5)
e= torch.cat((c,d),1)
e.shape


In [ ]:
(c-d)

In [ ]:
-1.5705-1.4329

In [ ]:
(c*d)

In [ ]:
-1.5705*1.4329

In [ ]:
-0.6561*-0.2841

In [ ]:
'''blocksize=2
k= ((li[block:block+blocksize], block) for block in range(0,len(li),blocksize))
for i,j in k:
    print(i,j)'''

In [ ]:
'''def worker(args):
        print(args[0])
        print(args[1])'''

In [ ]:
p= np.random.randn(1,5)
p

# comparing different losses

In [ ]:
import torch
import torch.nn as nn


In [ ]:
loss = nn.NLLLoss()
klloss= nn.KLDivLoss(reduction='batchmean')

In [ ]:
pred = torch.tensor(([0.21, 0.79], [0.68, 0.32]), dtype = torch.float)

In [ ]:
pred

tensor([[0.2100, 0.7900],
        [0.6800, 0.3200]])

In [ ]:
target1 = torch.tensor(([0.2,0.8],[0.7,0.3]),dtype = torch.float)
target=torch.max(target1,1)[1]
print(target1)
print(target)


tensor([[0.2000, 0.8000],
        [0.7000, 0.3000]])
tensor([1, 0])


In [ ]:
def cross_entropy(pred,target):
  output = loss(torch.log(pred), target)
  print(output)

In [ ]:
def crosse_wo_onehot(pred,target1):
  print((-(target1[0,0]*pred[0,0 ]) - (target1[0,1]*pred[0,1 ]) - (target1[1,0]*pred[1,0 ]) - (target1[1,1]*pred[1,1 ])) / 2)

In [ ]:
crosse_wo_onehot(pred,target1)

tensor(0.5563)


In [ ]:
pred = torch.tensor(([0.25, 0.75], [0.65, 0.35]), dtype = torch.float)

In [ ]:
cross_entropy(pred,target)
pred_log= torch.log(pred)
crosse_wo_onehot(pred_log,target1)
print(klloss(pred_log,target1))

tensor(0.3592)
tensor(0.5619)
tensor(0.0063)


In [ ]:
!pip3 install fasttext

In [ ]:
import fasttext

In [ ]:
!pip3 install sh

     |████████████████████████████████| 40kB 2.8MB/s 


In [ ]:
from sh import gunzip

In [ ]:
gunzip('cc.en.300.bin.gz')

In [ ]:
ft = fasttext.load_model('cc.en.300.bin')

ValueError: ignored

In [ ]:
ft.get_word_vector('hello')

array([ 1.57576188e-01,  4.37820926e-02, -4.51271934e-03,  6.65931404e-02,
        7.70346820e-02,  4.85855248e-03,  8.19822028e-03,  6.52402919e-03,
        9.25899856e-03,  3.53899002e-02, -2.31395271e-02, -4.91807126e-02,
       -8.32642540e-02,  1.56014524e-02,  2.54856616e-01,  3.45423706e-02,
       -1.07451361e-02, -7.80188590e-02, -7.08099529e-02,  7.62385577e-02,
       -6.09613657e-02,  4.48625796e-02, -7.29744136e-02,  1.30583309e-02,
        3.14881057e-02, -3.10055036e-02,  1.66004002e-02,  1.74405202e-02,
       -7.35838860e-02,  1.18252613e-01, -1.21330231e-01, -4.09253240e-02,
        2.93969568e-02,  4.84445989e-02, -1.33816330e-02, -1.74765270e-02,
        7.51308873e-02,  9.97046307e-02, -4.00476977e-02,  4.05735290e-03,
       -7.21896589e-02, -4.43356819e-02, -1.22628408e-03,  7.56693557e-02,
        3.98401320e-02,  3.22643593e-02,  1.95914153e-02,  4.68016043e-02,
       -1.46228177e-02,  1.12967767e-01,  3.15065160e-02, -1.02312110e-01,
        1.58124104e-01, -

In [ ]:
ft